# <span style="color:green">Name: Shivam Soni</span>

## <span style="color:green">SISU coding solution - Data engineering</span>

###  <span style="color:green"> 1. Libraries imported</span> 

In [ ]:
import pandas as pd
import numpy as np
import os as os
import datetime as dt

In [ ]:
# Importing the csv file as pandas data frame
transaction_df = pd.read_csv("Transaction.csv")

### <span style="color:green"> 2. Exploratory data analysis and Data cleaning </span> 

In [ ]:
transaction_df.shape

This means that there are __**30320**__ rows or records and __**19**__ columns in the dataframe.

Here we the following to clean the data:
1. Delete xyz and nan values records for Account Status
2. Delete nan value records for Property Type
3. Check the format of each row, if anamoly detected then convert into the right format
4. Remove the unnamed columns from the tail end of the dataframe

In [ ]:
# Last two columns are being read as Unnamed:17 & Unnamed:18 which are deleted below
del transaction_df[transaction_df.columns[len(transaction_df.columns)-2]]
del transaction_df[transaction_df.columns[len(transaction_df.columns)-1]]

#Removing unnecessary spaces from the header
transaction_df.columns = transaction_df.columns.str.replace(' ','')

#Loading all the column names into a list
column_list = list(transaction_df.columns)

In [ ]:
#Useful columns into a list
len(column_list)

In [ ]:
#Declaring dictionary to store unique values of relevant columns
dict_of_unique_values = {}
for number in range(len(column_list)-1):
    req_no = column_list[number] 
    dict_of_unique_values[req_no] = list(transaction_df[req_no].unique())
    
#Displaying unique values for relevant columns
for number in range(len(column_list)-1):
    if not number in [0,2,7,10,13,14]: 
        print(column_list[number]+" ---> "+str(dict_of_unique_values[column_list[number]]))

In [ ]:
#Storing the indexes of na values in Account status and property type
index_na_acc_status = list(np.where(transaction_df['Accountstatus'].isna()))
index_na_prp_status = list(np.where(transaction_df['PropertyTYPE'].isna()))

#Converting the indexes into a list
index_na_acc_status = list(index_na_acc_status)
index_na_prp_status = list(index_na_prp_status)


temp_len = len(index_na_acc_status[0])
temp_len_1 = len(index_na_prp_status[0])

del_index_list = []
    
for i in range(temp_len):
    del_index_list.append(index_na_acc_status[0][i])
for i in range(temp_len_1):
    del_index_list.append(index_na_prp_status[0][i])
    
transaction_df = transaction_df.drop(del_index_list)

In [ ]:
#This code tab is declaring a function to get the index number of the columns

#Function to find index of the column
def find_index(string):
    return int(column_list.index(string))

#Function to convert string columns into date stamp
def convert_to_date(column_name):
    transaction_df[column_name] = pd.to_datetime(transaction_df[column_name])
    return transaction_df

In [ ]:
len(transaction_df)

In [ ]:
#Total number of agents
no_of_agents = len(dict_of_unique_values[column_list[find_index("AgentID")]])
print("There are "+str(no_of_agents)+" Agents in total.")

### <span style="color:green"> 3. Writing records in JSON one at a time and creating batch files for every 1000 records </span> 

In [ ]:
# Deleting any existing files 
for i in range(round(((len(transaction_df)+4)/1000)+2)):
    if os.path.isfile("{}.json".format(i+1)):
        os.remove("{}.json".format(i+1))

In [ ]:
#Task number 2,3,4 & 5 are done in this block of code.
n =1

#Creating new files as per instructions
for i in transaction_df.index:
    
    #Watching the above task in the output terminal
    print("{}\n".format(i)+  transaction_df.loc[i].to_json()+ "\n\n" )
    
    #Writing the data in JSON one record at a time
    #Naming it 1.json, 2.json according to the batch of 1000 records
    transaction_df.loc[i].to_json("{}.json".format(n))
    
    #Creating a JSON file for every 1000 records
    if (int(i/1000) == n):
        n = n + 1
        print(n)

### <span style="color:green"> 3. Converting the date columns into date format and calculating the response time to show list of post codes based on fastest response </span> 

In [ ]:
#Declaring the list of columns to be converted from string to date
str_to_date_list = ["RequestDate","lastUpdatedDate","ImplementedDate"]

#Using the above declared function to convert the declared list of columns to date
for n in str_to_date_list:
    convert_to_date(n)

#Adding a new row to calculate the response time by the agent
transaction_df['ResponseTime'] = transaction_df['RequestDate'] - transaction_df['ImplementedDate']

#Getting just the number of days as a difference betweeen Request date and implemented date
transaction_df["ResponseTime"] = [transaction_df.iloc[i]["ResponseTime"].days for i in range(len(transaction_df))]
transaction_df["ResponseTime"] = [abs(transaction_df.iloc[i]["ResponseTime"]) for i in range(len(transaction_df))]

#Displaying list of codes based on fastest response which is the minimum number of days taken from requested to implemented
transaction_df.groupby(by="PostCode")["ResponseTime"].min()

### <span style="color:green"> 4. Calculating and displaying top agents based on Post codes and Amount </span> 

In [ ]:
#Displaying top agents with respect to Post code and Agent ID

temp_df = transaction_df.groupby(by=["PostCode","AgentID"])["$Amount"].sum()
top_agents = temp_df.loc[temp_df.groupby(level=0).idxmax()]
print(top_agents)